In [3]:
import numpy as np
import matplotlib.pyplot as plt
import csv

#Root of jupyter notebook is /projects/ps-gymreklab/resources/ukbiobank

In [4]:
def loadFreqs(freqLocation, identifyingVCFLocation, cNum, vcfHeaderLines):
    lociToFreqs = {}
    with open('{}/chr{}.afreq'.format(freqLocation, cNum)) as freqFile, \
            open('{}/chr{}.vcf'.format(identifyingVCFLocation, cNum)) as vcfFile:
        freqReader = csv.reader(freqFile, delimiter='\t')
        vcfReader = csv.reader(vcfFile, delimiter='\t')

        #skip header lines
        next(freqReader)
        for _ in range(vcfHeaderLines):
            next(vcfReader)


        count = 0
        for freqRow, vcfRow in zip(freqReader, vcfReader):
            count += 1
            if freqRow[1] != vcfRow[2]:
                print("Mismatched vcf and afreq IDs at variant {}".format(count))
                1/0
            variantKey = tuple(vcfRow[i] for i in[0,1,3,4])
            if variantKey in lociToFreqs:
                print("Already encountered variant {}".format(count))
                1/0
            lociToFreqs[variantKey] = freqRow[4]

            if count % 100000 == 0:
                print("Count {}k   ".format(count/1000), end='\r')
            
    return lociToFreqs

In [ ]:
#set({'a':4,'c':7}.keys()).intersection(set({'a':1, "b":2}.keys()))
pass

In [20]:
variantCount = 0
xs = np.zeros(1000000)
ys = np.zeros(1000000)

#for cNum in range(1,23):
for cNum in range(22,23):
    snpstrFreqs = loadFreqs("/projects/ps-gymreklab/resources/ukbiobank/pre_imputation_qc/allele_freqs/snpstr",
                            "/projects/ps-gymreklab/resources/ukbiobank/snpstr/vcf_1_sample", cNum, 11)
    hapFreqs = loadFreqs("/projects/ps-gymreklab/resources/ukbiobank/pre_imputation_qc/allele_freqs/hap",
                            "/projects/ps-gymreklab/resources/ukbiobank/original/vcf_1_sample/hap", cNum, 6)
    commonKeys = set(snpstrFreqs.keys()).intersection(set(hapFreqs.keys()))
    for key in commonKeys:
        xs[variantCount] = snpstrFreqs[key]
        ys[variantCount] = hapFreqs[key]
        variantCount += 1


In [ ]:
plt.hexbin(xs, ys, cmap='Blues')
plt.colorbar(label='count in bin')

In [28]:
for i, key in enumerate(snpstrFreqs.keys()):
    if i==10:
        break
    print(key, snpstrFreqs[key])
print("Hap")
for i, key in enumerate(hapFreqs.keys()):
    if i==10:
        break
    print(key, hapFreqs[key])

('22', '16050115', 'G', 'A') 0.00638978
('22', '16050213', 'C', 'T') 0.00758786
('22', '16050527', 'C', 'A') 0.000199681
('22', '16050607', 'G', 'A') 0.000998403
('22', '16050783', 'A', 'G') 0.00778754
('22', '16050840', 'C', 'G') 0.00519169
('22', '16050847', 'T', 'C') 0.00119808
('22', '16050984', 'C', 'G') 0.00399361
('22', '16051075', 'G', 'A') 0.00139776
('22', '16051246', 'G', 'A') 0.000199681
Hap
('22', '16057417', 'T', 'C') 0.897911
('22', '16495833', 'A', 'C') 0.921837
('22', '16595552', 'A', 'C') 0.959743
('22', '16870425', 'T', 'C') 0.6884
('22', '16880098', 'A', 'G') 0.941511
('22', '16888577', 'A', 'G') 0.709247
('22', '16890136', 'A', 'G') 0.977362
('22', '16915685', 'A', 'G') 0.99911
('22', '16952830', 'C', 'G') 0.984883
('22', '17054795', 'A', 'G') 0.874829


(0, 0)
(1, 1)
(2, 2)
